### 1. Подключение библиотек
Наличие большого числа разносторонних библиотек - одно из ключевых преимуществ языка Python  
Среди огромного количества библиотек стоит выделить следующие:
* numpy - библиотека для работы с массивами/матрицами
* pandas - библиотека для работы с таблицами
* RDKit - библиотека для работы с молекулами и хим. сущностями
* matplotlib - библиотека для визуализации
* psi4 - квантовые расчеты
* openmm - мол. динамика
* pickle - библиотека для сохранения объектов в бинарных файлах
* os - для работы с файловой системой

In [ ]:
### Pandas понадобится нам для работы с данными
import pandas as pd
### Различные подмодули из rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem.Draw import IPythonConsole
### для сохранения в файл
import pickle
### для работы с файловой системой
import os
### для отрисовки
from matplotlib import pyplot as plt
from matplotlib.image import NonUniformImage
### Для работы с массивами
import numpy as np
### Импорты для анализа потраченной памяти
from sys import getsizeof
import psutil

### Переменная, указывающая папку с нашими файлами
data_folder = "/home/alex/ml_lectures"

### 2. Загрузка исходных данных
#### Что представляют данные в датасете
    I.  Property  Unit         Description
    --  --------  -----------  --------------
     1  tag       -            "gdb9"; string constant to ease extraction via grep
     2  index     -            Consecutive, 1-based integer identifier of molecule
     3  A         GHz          Rotational constant A
     4  B         GHz          Rotational constant B
     5  C         GHz          Rotational constant C
     6  mu        Debye        Dipole moment
     7  alpha     Bohr^3       Isotropic polarizability
     8  homo      Hartree      Energy of Highest occupied molecular orbital (HOMO)
     9  lumo      Hartree      Energy of Lowest occupied molecular orbital (LUMO)
    10  gap       Hartree      Gap, difference between LUMO and HOMO
    11  r2        Bohr^2       Electronic spatial extent
    12  zpve      Hartree      Zero point vibrational energy
    13  U0        Hartree      Internal energy at 0 K
    14  U         Hartree      Internal energy at 298.15 K
    15  H         Hartree      Enthalpy at 298.15 K
    16  G         Hartree      Free energy at 298.15 K
    17  Cv        cal/(mol K)  Heat capacity at 298.15 K

In [ ]:
### В прошлой раз мы разобрались с загрузкой. Просто выгружаем данные из csv
df1 = pd.read_csv(os.path.join(data_folder,"qm9.csv"))

In [ ]:
### удалем столбец mol_id

### Проверка на то, что столбец -- это обычная индексация без проблемных мест
check_val = (df1["mol_id"].apply(lambda x: int(x[4:]) - 1) - np.arange(df1.shape[0])).abs().sum()
print( "mol_id просто индексатор" if check_val==0 else "С mol_id что-то интересное")
### С чистой совестью можем удалять mol_id
### inplace указывает на то, что мы просим сделать это на месте,
### а не возвращать копию. См. help(df1.drop)
df1.drop(columns="mol_id", inplace=True)
df1

In [ ]:
### Очень важно при отборе фичей для ML избегать сильных корреляций.
### Взглянем на парные корреляции по kendall внутри датасета
corr_map = df1[list(df1)[1:]].corr(method='kendall')
corr_map

In [ ]:
corr_np = np.triu(corr_map.to_numpy(), k=1)
idx = np.unravel_index(corr_np.argmax(), corr_np.shape)
print("Максимальная корреляция между столбцами {} и {} и равна {:.10f}".format( 
            list(corr_map)[idx[0]], list(corr_map)[idx[1]], corr_map.iloc[idx]))

При построении сложных моделей __обычно отказываются от столбцов при корелляции более 0.98 или 0.95.__  
Отметим, что более многомерные корелляции здесь не проверяются. Не сложно понять, что gap = lumo - homo  
Но такого мы таким простым способом не выявим, к сожалению  
**<u> ДЗ: почистить датасет, чтобы максимальная корреляция по Кендаллу между столбцами была меньше 0.97</u>**

In [ ]:
### ДЗ, чистим матрицу
### Берез матрицу абсолютных значений парных корелляций
cor_matrix = df1.iloc[:,1:].corr(method='kendall').abs()
### Выбираем верхний треугольник данной матрицы (без диагонали)
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(bool))
### Выбирвем столбцы для удаления
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.97)]
### Выкидываем столбцы, и создаем новое представление df1_nocorr
df1_nocorr = df1.drop(columns=to_drop)
### Выкидываем столбец gap, используя эмпирические знания о том, что gap = lumo - homo
df1_nocorr = df1_nocorr.drop(columns="gap")
### Посмотрим на результат
df1_nocorr

### 3. Предсказываем $r^2$
#### __3.1 Разделение данных__

In [ ]:
### Стандартная функция для разделения датасета
from sklearn.model_selection import train_test_split

Итак, в следующих ячейках мы воспользуемся функцией __train_test_split дважды подряд__  
Почему и что важно помнить!!! У нас есть 2 базыовые части датасета:
* __Тренировочная часть__, доступная нам в ходе поиска и отпимизации моделей машинного обучения.
* __Тестовая часть__, которую __нельзя использовать ни для сравнения моделей, ни для их оптимизации__.  
  Это часть данных используется лишь раз при представлении финальных результатов на выбранной, готовой модели.  
  Эти данные мы отделим в самом начале и не будем к ним прикасаться почти до конца. Назовем их __outer_test__.  
  
Для того, чтобы искать лучшую модель, мы отделим от тренировочного еще кусок,  
он будет указан здесь как __test__ лишь в силу исторических причин, но по факту и духу - __это валидационный датасет__

In [ ]:
### Разделяем наш массив на тренировочную+валидационную и тестовую выборки
### test_size указывает, что мы хотим 15% данных в качестве тестовых
### random_state позволяет указывать выбранное случайное значение при изначальном
###    перемешивании строк для воспроизводимости результатов
train_and_valid, outer_test = train_test_split(df1_nocorr, test_size = 0.15, random_state=1234)

### Взглянем на train
train_and_valid

In [ ]:
### 10% на валидацию
train, test = train_test_split(train_and_valid, test_size = 0.1, random_state=1234)

In [ ]:
### Теперь нам нужно выделить X, y -- данные на которых базируются 
### предсказания и данные, которые мы хотим предсказывать
col_name = "r2"
X_train = train[[x for x in train if x!="smiles" and x!=col_name]]
y_train = train[col_name]
X_test = test[[x for x in test if x!="smiles" and x!=col_name]]
y_test = test[col_name]

In [ ]:
### Разделить данные можно также в другом порядке
X = train_and_valid[[x for x in df1_nocorr if x!="smiles" and x!=col_name]]
y = train_and_valid[col_name]
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(X, y, test_size = 0.1, random_state=1234)

In [ ]:
### Проверим идентичность разбиений при смене порядка действий
### Отметим, что равенство достигается только при заданном random_state
print( "Разбиения идентичны" if (X_train.equals(X_train_v2) and X_test.equals(X_test_v2) \
                                 and y_train.equals(y_train_v2) and y_test.equals(y_test_v2)) \
      else "Разбиения НЕ ОДИНАКОВЫ !!!")

In [ ]:
### В самом простом случае за разбиением данных следует создание модели,
### ее обучение и использование для предсказаний с предварительным тестированием
### Восползуемся простейшим линейным регрессором
from sklearn.linear_model import LinearRegression as LR
### Стандартные метрики для анализа качества регрессионных моделей
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
### создаем объект класса линейного регрессора
lr = LR()
### Обучаем на тренировочных данных
lr.fit(X=X_train, y=y_train)
### Сохраняем предсказания
preds = lr.predict(X_test)
print("Результаты на тесте: MAE = {:.2f}, RMSE = {:.2f}".
      format( mean_absolute_error(preds, y_test),
              mean_squared_error(preds, y_test) ))

### Для одиночной модели, эти данные чуть более чем бесполезны
### Куда приятнее взглянуть на относительные погрешности, 
### но с ними нужно быть крайне аккуратными и следить, на что вы делите
print( "В столбце r2 ровно {} строк со значением меньше 0.1".
      format(df1_nocorr[df1_nocorr['r2']<0.1].shape[0]))
print( "Средняя относительная погрешность = {:.2f}%".format( ((preds-y_test)/y_test).abs().mean()*100 ))

In [ ]:
def print_metrics(preds,yt, check_col=None, df=None):
    print("Результаты на тесте: MAE = {:.2f}, RMSE = {:.2f}".
          format( mean_absolute_error(preds, yt),
                  mean_squared_error(preds, yt) ))
    if check_col is not None:
    ### Для одиночной модели, эти данные чуть более чем бесполезны
    ### Куда приятнее взглянуть на относительные погрешности, 
    ### но с ними нужно быть крайне аккуратными и следить, на что вы делите
        print( "В столбце "+check_col+" ровно {} строк со значением меньше 0.1".
              format(df[df[check_col]<0.1].shape[0]))
    print( "Средняя относительная погрешность = {:.2f}%".format( ((preds-yt)/yt).abs().mean()*100 ))
    

### Стоит оформить это в функцию:
def fit_pred(X,Xt,y,yt,model, df=df1_nocorr, col='r2'):
    ### Обучаем на тренировочных данных
    model.fit(X=X, y=y)
    ### Сохраняем предсказания
    preds = model.predict(Xt)
    print_metrics(preds,yt,check_col=col,df=df)
    return model

In [ ]:
### Взглянем на коэффициенты линейной модели
print("Linear model coefficients:\n")
for i in range(X_test.shape[1]):
    print(" {:>7} = {:>10.4f} ".format(X_test.columns[i],  lr.coef_[i]))

Без нормализации исходного вектора, эти числа не в полной мере отражают важность признаков  
Попробуем, по-быстрому, посмотреть на результат SHAP-анализа.  
На графике ниже показано насколько сильно тот или иной вклад фичи меняет вклад для каждой точки.

In [ ]:
### библиотека для shap-анализа
import shap
explainer = shap.Explainer(lr.predict, X_test)
### Используем только 2000 строк, ибо долго
shap_values = explainer(X_train.head(2000))
shap.plots.beeswarm(shap_values)

Видим, что можно выкинуть колонку А и предсказание не поменяется.

In [ ]:
### Выкинем колонку А и проверим предсказание
_ = fit_pred(X_train.drop(columns='A'),X_test.drop(columns='A'),y_train,y_test, lr)

__Важное замечание__  
Линейный регрессор - простая, быстрая и пригодная модель для подобного анализа.  
Однако стоит помнить, данные результаты -- это  не гарантия того,  
что выкинутая фича не окажется полезной в другой модели.

In [ ]:
### Импорт из sklearn различных полезностей
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, ConstantKernel
# from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from xgboost import XGBRegressor
from copy import deepcopy as cp

In [ ]:
### offtopic: класс для удобного выделения текста в print
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
### Название классов моделей
names_solvers = ["XGB", "KNeighborsRegressor", "SVR_Sigmoid",
                 "Linear_Regression", "MLP"]
### Генерируем объекты классов
xgb = XGBRegressor()
knn = KNeighborsRegressor()
svr_sigma = SVR(kernel="sigmoid", max_iter=400, C=100 ,cache_size=1000)
lr = LR()
ml = MLPRegressor(hidden_layer_sizes=(64,32,16,8), activation='relu', max_iter=200)
### Заносим их в массив
solvers = [xgb, knn, svr_sigma, lr, ml ]

In [ ]:
### 3 цикла. Внутри каждого цикла запускаем все модели
### Обратите внимание на использование deepcopy
### Не самое удачное решение, но в ином случае модель во втором цикле
###   уже окажется обученной ((

### На обычном нетронутом датасете 
print(color.BOLD + "Без изменений датасета" + color.END)
for solv, name in zip(solvers, names_solvers):
    print("Start with "+name)
    solver_copy = cp(solv)
    preds = solver_copy.fit(X_train, y_train).predict(X_test)
    print_metrics(preds,y_test)
    print()
print()
### Перед моделью стоит нормализатор. Он переводит столбцы в более приглядный
###   для некоторых моделей вид. 
print(color.BOLD + "С использованием перенормировки" + color.END)
for solv, name in zip(solvers, names_solvers):
    print("Start with "+name)
    solver_copy = cp(solv)
    model = make_pipeline(StandardScaler(), solver_copy)
    preds = model.fit(X_train, y_train).predict(X_test)
    print_metrics(preds,y_test)
    print()
print()
### Теперь не только нормализуются данные, но и генерируются квадраты фичей
print(color.BOLD + "С использованием перенормировки и квадратичных фичей" + color.END)
for solv, name in zip(solvers, names_solvers):
    print("Start with "+name)
    solver_copy = cp(solv)
    model = make_pipeline(StandardScaler(), PolynomialFeatures(), solver_copy)
    preds = model.fit(X_train, y_train).predict(X_test)
    print_metrics(preds,y_test)
    print()

__<u> ДЗ: </u> на примере столбца u0_atom показать как работает StandardScaler и PolynomialFeatures (ответ с графиками или гистограммами)__

In [ ]:
### Давайте взглянем какие фичи XGBoost выделяет как важные
### Обучим модель и восползуемся переменной feature_importances_
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
print_metrics(y_test,preds)
ax1 = plt.subplot()
ax1.set_xticks(range(len(xgb.feature_importances_)))
plt.bar(range(len(xgb.feature_importances_)), xgb.feature_importances_)
ax1.set_xticklabels(list(X_train))
plt.show()

In [ ]:
#### Что будет если взять только пару топовых фичей для XGBoost
arr = []
for feature in X_train.iloc[:,xgb.feature_importances_.argsort()[::-1][:5]].columns:
    arr.append(feature)
    xgb = XGBRegressor()
    preds = xgb.fit(X_train[arr], y_train).predict(X_test[arr])
    print("XGB with {} top features: ".format(len(arr)) + " ".join(arr))
    print_metrics(y_test,preds)
    print()

In [ ]:
### Если поменять random_state, то результаты на валидационных датасетах
###  могут отличаться друг от друга.
### Исходно было:
# Результаты на тесте: MAE = 10.41, RMSE = 191.96
# Средняя относительная погрешность = 0.87%
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, y, test_size = 0.1, random_state=432101234)
xgb = XGBRegressor()
xgb.fit(X_train_new, y_train_new)
preds = xgb.predict(X_test_new)
print_metrics(y_test_new,preds)

Чтобы иметь __статистически сформулированный ответ о точнотсти предсказания__  
можно воспользоваться Кросс-Валидацией с повторением

In [ ]:
### Кросс-валидация с повторением
### Прекрасный код, почти не тронут из мануала sklearn

from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
 
# evaluate a model with a given number of repeats
def evaluate_model(X, y, model_func, repeats=3):
    # prepare the cross-validation procedure
    model = model_func()
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
    return scores

# configurations to test
repeats = 3
results = dict()
for model_func,name in zip([XGBRegressor, KNeighborsRegressor],["XGBoostRegr", "KNeighborsRegressor"]):
    # evaluate using a given number of repeats
    scores = evaluate_model(X, y, model_func, repeats=repeats)
    # summarize
    print(name)
    print('>%d mean=%.4f se=%.3f' % (repeats, mean(scores), sem(scores)))
    print()
    # store
    results[name] = scores

In [ ]:
plt.boxplot(results.values(), labels=results.keys(), showmeans=True)
plt.show()

__<u> ДЗ: </u>__ Выше не оказалось MLP по 2 причинам:
1. MLP очень долго считается.
2. MLP нужны аргументы.  

Задание в том, чтобы, сохраняя передачу в evaluate_model класса модели,  
занести вовнутрь нужные параметры и указать их при инициализации объекта модели.  
Подсказка: смотрите в сторону \*args и \*\*kargs

__Почему стоит избавляться от скореллированных столбцов__  
Рассмотрим линейную модель со всеми столбцами

In [ ]:
### Берем все столбцы
### И делем все при тех же random_state (и тем же количеством строк)
train_and_valid_full, outer_test_full = train_test_split(df1, test_size = 0.15, random_state=1234)
X_full = train_and_valid_full[[x for x in df1 if x!="smiles" and x!=col_name]]
y_full = train_and_valid_full[col_name]
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y_full, test_size = 0.1, random_state=1234)

lr = LR()
lr = fit_pred(X_train_full, X_test_full, y_train_full, y_test_full, lr)
### Взглянем на коэффициенты линейной модели
print("Linear model coefficients:\n")
for i in range(X_test_full.shape[1]):
    print(" {:>10} = {:>15.4f} ".format(X_test_full.columns[i],  lr.coef_[i]))
### библиотека для shap-анализа
import shap
explainer = shap.Explainer(lr.predict, X_test_full)
### Используем только 2000 строк, ибо долго
shap_values = explainer(X_train_full.head(2000))
shap.plots.beeswarm(shap_values)

Вряд ли такие коэффициенты -- это наш эталон устойчивой системы.  
Стоит аккуратнее относится к этим результатом, даже если они стали лучше  
При этом xgb не изменилась практически

In [ ]:
xgbr = XGBRegressor()
xgbr = fit_pred(X_train_full, X_test_full, y_train_full, y_test_full, xgbr)

### Финальные результате на внешнем тесте

In [ ]:
xgbr = XGBRegressor()
print_metrics( xgbr.fit(X,y).predict(
    outer_test[[x for x in outer_test if x!='smiles' and x!='r2']]), outer_test['r2'])